<a href="https://colab.research.google.com/github/Retr0r0cket/Wargame-Tank-Identifier/blob/main/Wargame_Tank_Identifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Image Recognition Model

Screenshot to Clipboard